In [1]:
from tkinter import *
from tkinter import filedialog

In [ ]:
class VM:
    def __init__(self):
        self._registers = { 'i':0, 's':-1 }
        self._stack = []
        self._prog = []
        self._state = False
        self.__exec = {
            'LDC': self._LDC,
            'LDV': self._LDV,
            'ADD': self._ADD,
            'SUB': self._SUB,
            'MULT': self._MULT,
            'DIVI': self._DIVI,
            'INV': self._INV,
            'AND': self._AND,
            'OR': self._OR,
            'NEG': self._NEG,
            'CME': self._CME,
            'CMA': self._CMA,
            'CEQ': self._CEQ,
            'CDIF': self._CDIF,
            'CMEQ': self._CMEQ,
            'CMAQ': self._CMAQ,
            'START': self._START,
            'HLT': self._HLT,
            'STR': self._STR,
            'JMP': self._JMP,
            'JMPF': self._JMPF,
            'NULL': self._NULL,
            'RD': self._RD,
            'PRN': self._PRN,
            'ALLOC': self._ALLOC,
            'DALLOC': self._DALLOC,
            'CALL:': self._CALL,
            'RETURN': self._RETURN
        }
        
    def _incI(self, x):
        self._registers['i'] += 1
        
    def _decI(self, x):
        self._registers['i'] -= 1
        
    def _incS(self, x):
        self._registers['s'] += 1
        
    def _decS(self, x):
        self._registers['s'] -= 1
        
    def _getI(self, x):
        return self._registers['i']
        
    def _getS(self, x):
        return self._registers['s']
    
    def _parseInt(x, d=0):
        return int(x[d])
    
    def _parseLabel(x): # TODO implement this mess
        for idx, line in enumerate(self._prog):
            if x == line[0]:
                return idx
        return 0
    
    def _push(x, d=0):
        self._stack[self._getS()+d] = x
    
    def _pushAt(x, p):
        self._stack[p] = x
        
    def _fetch(x):
        return self._stack[x]
    
    def _pop(d=0):
        return self._stack[self._getS()+d]
    
    def _true(d=0):
        return self._pop(d=d) == 1
    
    def _false(d=0):
        return self._pop(d=d) == 0
        
    def _LDC(self, x):
        self._incS()
        self._push(self._parseInt(x))
        
    def _LDV(self, x):
        self._incS()
        self._push(self._fetch(self._parseInt(x)))
        
    def _ADD(self, x):
        self._push(self._pop(d=-1) + self._pop(), d=-1)
        self._decS()
        
    def _SUB(self, x):
        self._push(self._pop(d=-1) - self._pop(), d=-1)
        self._decS()
        
    def _MULT(self, x):
        self._push(self._pop(d=-1) * self._pop(), d=-1)
        self._decS()
        
    def _DIVI(self, x):
        self._push(self._pop(d=-1) // self._pop(), d=-1)
        self._decS()
        
    def _INV(self, x):
        self._push(-1 * self._pop())
        
    def _AND(self, x):
        self._push(1 if self._true() and self._true(d=-1) else 0, d=-1)
        self._decS()
        
    def _OR(self, x):
        self._push(1 if self._true() or self._true(d=-1) else 0, d=-1)
        self._decS()
        
    def _NEG(self, x):
        self._push(1 - self._pop())
        
    def _CME(self, x):
        self._push(1 if self._pop(d=-1) < self._pop() else 1, d=-1)
        self._decS()
        
    def _CMA(self, x):
        self._push(1 if self._pop(d=-1) > self._pop() else 1, d=-1)
        self._decS()
        
    def _CEQ(self, x):
        self._push(1 if self._pop(d=-1) == self._pop() else 1, d=-1)
        self._decS()
        
    def _CDIF(self, x):
        self._push(1 if self._pop(d=-1) != self._pop() else 1, d=-1)
        self._decS()
        
    def _CMEQ(self, x):
        self._push(1 if self._pop(d=-1) <= self._pop() else 1, d=-1)
        self._decS()
        
    def _CMAQ(self, x):
        self._push(1 if self._pop(d=-1) >= self._pop() else 1, d=-1)
        self._decS()
        
    def _START(self, x):
        self._registers['s'] = -1
        
    def _HLT(self, x):
        self.state = False
        
    def _STR(self, x):
        self._stack[self._parseInt(x)] = self._pop()
        self._decS()
        
    def _JMP(self, x):
        self._registers['i'] = self._parseInt(x)
        
    def _JMPF(self, x):
        self._registers['i'] = self._parseInt(x) if self._false() else self._registers['i'] + 1
        
    def _NULL(self, x):
        pass
    
    def _RD(self, x):
        self._incS()
        self._push(int(input(">"))) #TODO add input logic
        
    def _PRN(self, x):
        print(self._pop()) #TODO add output logic
        self._decS()
        
    def _ALLOC(self, x):
        m = self._parseInt(x)
        n = self._parseInt(x, d=1)
        for k in range(n):
            self._incS()
            self._push(self._fetch(m + k))
        
    def _DALLOC(self, x):
        m = self._parseInt(x)
        n = self._parseInt(x, d=1)
        for k in range(n-1, -1, -1):
            self._pushAt(self._pop(), m + k)
            self._decS()
            
    def _CALL(self, x):
        self._incS()
        self._push(self._getI() + 1)
        self._registers['i'] = self._parseLabel(x)
        
    def _RETURN(self, x):
        self._registers['i'] = self._pop()
        self._decS()
        
    def toString(self):
        return "VM [i={}, s={}]".format(self._registers['i'], self._registers['s']) + "["+",".join(self._stack)+"]"
        
    def loadProgram(self, program):
        self._prog = program
        
    def execute(self):
        self._state = True
        while self._state:
            self.step()
            
    def step(self):
        if not self._state:
            return 0
        cmd = self._prog[self._getI()]
        aux = cmd[2] if len(cmd) > 2 else None
        args = cmd[1] if len(cmd) > 1 else None
        cmd = cmd[0]
        if cmd in self.__exec:
            self.__exec[cmd](args)
        else:
            self.__exec[args](aux)

In [3]:
class ProgramTextView(Frame):
    def __init__(self, master, **kwargs):
        Frame.__init__(self, master)
        
        # Default width
        if 'width' not in kwargs:
            kwargs['width'] = 30
        w = kwargs['width']    
        
        kwargs['width'] = 5
        self.lineIndicator = Text(self, **kwargs)
        self.lineIndicator.pack(side=LEFT, fill=BOTH)
        kwargs['width'] = w - 5
        self.programView = Text(self, **kwargs)
        self.programView.pack(side=LEFT, fill=BOTH, expand=True)
        self.scrollbar = Scrollbar(self)
        self.scrollbar.pack(side=RIGHT, fill=Y)
        
        self.scrollbar['command'] = self.on_scrollbar
        self.lineIndicator['yscrollcommand'] = self.on_textscroll
        self.programView['yscrollcommand'] = self.on_textscroll
        
    def on_scrollbar(self, *args):
        self.lineIndicator.yview(*args)
        self.programView.yview(*args)
        
    def on_textscroll(self, *args):
        self.scrollbar.set(*args)
        self.on_scrollbar('moveto', args[0])
        
    def __clear(self):
        self.programView.delete("1.0", END)
        self.lineIndicator.delete("1.0", END)
        
    def writeProgramField(self, program):
        self.__clear()
        lineNo = len(program)
        print("LineNo = {}".format(lineNo))
        lines = "\n".join([" ".join(["[{}]".format(element) for element in line.split()]) for line in program])
        self.programView.insert(END, lines)
        self.lineIndicator.insert(END, "\n".join([str(i + 1) for i in range(lineNo)]))
            

In [4]:
def tkExit():
    print("Called")
    root.quit
    root.destroy

In [5]:
def chooseFile():
    file_path = filedialog.askopenfilename()
    print("Usuario escolheu [{}]".format(file_path))
    prog = importProgramFile(file_path)
    programTextField.writeProgramField(prog)
    vm.loadProgram([line.split() for line in prog])

In [6]:
def importProgramFile(file_path):
    fd = open(file_path, "r")
    return fd.readlines()

In [7]:
def on_scrollbar(*args):
    lineIndicatorTextField.yview(*args)
    programTextField.yview(*args)

In [ ]:
vm = VM()

In [ ]:
root = Tk()
Label(root, text="Hello, Tkinter").grid(row=0)
Button(root, text="Escolher arquivo", width=25, height=5, bg="#88ccfc", command=chooseFile).grid(row=1)
programTextField = ProgramTextView(root, width=100)
programTextField.grid(row=2)

mainloop()

text scroll!
('0.0', '0.058823529411764705')
text scroll!
('0.0', '0.058823529411764705')
text scroll!
('0.0', '1.0')
text scroll!
('0.0', '1.0')
Usuario escolheu [/home/ivy/Downloads/progTeste.lpd]
LineNo = 38
text scroll!
('0.0', '0.631578947368421')
text scroll!
('0.0', '0.631578947368421')
text scroll!
('0.07739938080495357', '0.7089783281733746')
text scroll!
('0.07739938080495357', '0.7089783281733746')
text scroll!
('0.15479876160990713', '0.7863777089783281')
text scroll!
('0.15479876160990713', '0.7863777089783281')
text scroll!
('0.23219814241486067', '0.8637770897832817')
text scroll!
('0.23219814241486067', '0.8637770897832817')
text scroll!
('0.30959752321981426', '0.9411764705882353')
text scroll!
('0.30959752321981426', '0.9411764705882353')
text scroll!
('0.23219814241486067', '0.8637770897832817')
text scroll!
('0.23219814241486067', '0.8637770897832817')
text scroll!
('0.15479876160990713', '0.7863777089783281')
text scroll!
('0.15479876160990713', '0.7863777089783281

text scroll!
('0.1130030959752322', '0.7445820433436533')
text scroll!
('0.11609907120743033', '0.7476780185758514')
text scroll!
('0.11609907120743033', '0.7476780185758514')
text scroll!
('0.11919504643962849', '0.7507739938080495')
text scroll!
('0.11919504643962849', '0.7507739938080495')
text scroll!
('0.12074303405572756', '0.7523219814241486')
text scroll!
('0.12074303405572756', '0.7523219814241486')
text scroll!
('0.1238390092879257', '0.7554179566563467')
text scroll!
('0.1238390092879257', '0.7554179566563467')
text scroll!
('0.12693498452012383', '0.7585139318885449')
text scroll!
('0.12693498452012383', '0.7585139318885449')
text scroll!
('0.12848297213622292', '0.760061919504644')
text scroll!
('0.12848297213622292', '0.760061919504644')
text scroll!
('0.13157894736842105', '0.7631578947368421')
text scroll!
('0.13157894736842105', '0.7631578947368421')
text scroll!
('0.1346749226006192', '0.7662538699690402')
text scroll!
('0.1346749226006192', '0.7662538699690402')
text

text scroll!
('0.3173374613003096', '0.9489164086687306')
text scroll!
('0.3157894736842105', '0.9473684210526315')
text scroll!
('0.3157894736842105', '0.9473684210526315')
text scroll!
('0.3080495356037152', '0.9396284829721362')
text scroll!
('0.3080495356037152', '0.9396284829721362')
text scroll!
('0.30495356037151705', '0.9365325077399381')
text scroll!
('0.30495356037151705', '0.9365325077399381')
text scroll!
('0.3018575851393189', '0.93343653250774')
text scroll!
('0.3018575851393189', '0.93343653250774')
text scroll!
('0.29721362229102166', '0.9287925696594427')
text scroll!
('0.29721362229102166', '0.9287925696594427')
text scroll!
('0.29411764705882354', '0.9256965944272446')
text scroll!
('0.29411764705882354', '0.9256965944272446')
text scroll!
('0.29256965944272445', '0.9241486068111455')
text scroll!
('0.29256965944272445', '0.9241486068111455')
text scroll!
('0.28637770897832815', '0.9179566563467493')
text scroll!
('0.28637770897832815', '0.9179566563467493')
text scr

text scroll!
('0.0030959752321981426', '0.6346749226006192')
text scroll!
('0.0030959752321981426', '0.6346749226006192')
text scroll!
('0.0046439628482972135', '0.6362229102167183')
text scroll!
('0.0046439628482972135', '0.6362229102167183')
text scroll!
('0.010835913312693499', '0.6424148606811145')
text scroll!
('0.010835913312693499', '0.6424148606811145')
text scroll!
('0.015479876160990712', '0.6470588235294118')
text scroll!
('0.015479876160990712', '0.6470588235294118')
text scroll!
('0.02321981424148607', '0.6547987616099071')
text scroll!
('0.02321981424148607', '0.6547987616099071')
text scroll!
('0.02631578947368421', '0.6578947368421053')
text scroll!
('0.02631578947368421', '0.6578947368421053')
text scroll!
('0.030959752321981424', '0.6625386996904025')
text scroll!
('0.030959752321981424', '0.6625386996904025')
text scroll!
('0.034055727554179564', '0.6656346749226006')
text scroll!
('0.034055727554179564', '0.6656346749226006')
text scroll!
('0.05108359133126935', '0.

text scroll!
('0.29102167182662536', '0.9226006191950464')
text scroll!
('0.29102167182662536', '0.9226006191950464')
text scroll!
('0.21362229102167182', '0.8452012383900929')
text scroll!
('0.21362229102167182', '0.8452012383900929')
text scroll!
('0.13622291021671826', '0.7678018575851393')
text scroll!
('0.13622291021671826', '0.7678018575851393')
text scroll!
('0.058823529411764705', '0.6904024767801857')
text scroll!
('0.058823529411764705', '0.6904024767801857')
text scroll!
('0.0', '0.631578947368421')
text scroll!
('0.0', '0.631578947368421')
text scroll!
('0.07739938080495357', '0.7089783281733746')
text scroll!
('0.07739938080495357', '0.7089783281733746')
text scroll!
('0.15479876160990713', '0.7863777089783281')
text scroll!
('0.15479876160990713', '0.7863777089783281')
text scroll!
('0.23219814241486067', '0.8637770897832817')
text scroll!
('0.23219814241486067', '0.8637770897832817')
text scroll!
('0.30959752321981426', '0.9411764705882353')
text scroll!
('0.309597523219

text scroll!
('0.0030959752321981426', '0.6346749226006192')
text scroll!
('0.0030959752321981426', '0.6346749226006192')
text scroll!
('0.0046439628482972135', '0.6362229102167183')
text scroll!
('0.0046439628482972135', '0.6362229102167183')
text scroll!
('0.010835913312693499', '0.6424148606811145')
text scroll!
('0.010835913312693499', '0.6424148606811145')
text scroll!
('0.01238390092879257', '0.6439628482972136')
text scroll!
('0.01238390092879257', '0.6439628482972136')
text scroll!
('0.018575851393188854', '0.6501547987616099')
text scroll!
('0.018575851393188854', '0.6501547987616099')
text scroll!
('0.020123839009287926', '0.651702786377709')
text scroll!
('0.020123839009287926', '0.651702786377709')
text scroll!
('0.02321981424148607', '0.6547987616099071')
text scroll!
('0.02321981424148607', '0.6547987616099071')
text scroll!
('0.02631578947368421', '0.6578947368421053')
text scroll!
('0.02631578947368421', '0.6578947368421053')
text scroll!
('0.02786377708978328', '0.6594